# *1.创建一个项目*

在命令行（终端）中，cd（例如cd exam)到你想要用来保存代码的目录，然后运行如下命令：

In [ ]:
$ django-admin startproject project01

其目录结构：

In [ ]:
project01/
    manage.py
    project01/
        __init__.py
        settings.py
        urls.py
        wsgi.py

修改project01/project01/settings.py文件

In [ ]:
LANGUAGE_CODE = 'zh-hans'   #修改语言
TIME_ZONE = 'Asia/Chongqing'  #修改时区

## 1.1查看目录内容

cd 进入目录， 使用dir命令

## 1.2数据库的建立及更新

In [ ]:
$python manage.py migrate

## 1.3运行服务器

In [ ]:
$python manage.py runserver

默认为127.0.0.1:8000

### 1.3.1改IP地址或端口

In [ ]:
$python manage.py runserver 8080
$python manage.py runserver 0.0.0.0：8000

---

# *2.创建模型*

## 2.1创建应用

In [ ]:
$ python manage.py startapp polls

polls为应用名称，此次以创建一个投票系统为例

其目录结构为:

In [ ]:
polls/
    __init__.py
    admin.py
    migrations/
        __init__.py
    models.py
    tests.py
    views.py

## 2.2编辑模型

编辑polls/models.py文件

## 2.3激活模型

### 2.3.1编辑mysite/setting.py  

在INSTALLED_APPS下添加'polls'  

### 2.3.2迁移

In [ ]:
$ python manage.py makemigrations polls

### 2.3.3更新

In [ ]:
$ python manage.py migrate

### 2.3.4查看迁移内容

In [ ]:
$ python manage.py sqlmigrate polls 0001  

具体在polls/migrations/0001_initial.py  

---

# *3.Django的API(实例讲解)*

## 3.1调用Python shell

In [ ]:
$ python manage.py shell

或者设置DJANGO_SETTINGS_MODULE 环境变量为 mysite.settings，启动一个普通的Python shell，然后建立Django：  

In [ ]:
import django
django.setup()

## 3.2实例讲解

In [ ]:
from polls.models import Question, Choice
from django.utils import timezone

---

输出所有问题实例对象，这个功能是继承自django.db.models.Model

In [ ]:
Question.object.all()
->[]

---

创建一个问题实例

In [ ]:
q = Question(question_text="What's new?", publish_date=timezone.now())

---

保存至数据库，不会因退出Shell而消失，再次进入时，q变量消失

In [ ]:
q.save()

---

输出这个实例的ID，这个功能是继承自django.db.models.Model

In [ ]:
q.id
->1

---

访问属性

In [ ]:
q.question_text
-> "What's new?"
q.publish_date
-> datetime.datetime(2012, 2, 26, 13, 0, 0, 775217, tzinfo=<UTC>)

---

修改属性

In [ ]:
q.question_text = "What's up?"
q.save()

---

In [ ]:
Question.objects.all()
-> [<Question: Question object>]

给Question类和Choice类添加__str__()方法后可以改变输出

In [ ]:
def __str__(self):
    return self.question_text
Question.objects.all()
-> [<Question: What's up?>]

---

可以添加自定义方法  
如在Question类下添加

In [ ]:
def was_published_recently(self):
    return self.publish_date >= timezone.now() - datetime.timedelta(days=1)
q.was_published_recently()
-> True

---

访问数据库中的问题

In [ ]:
Question.objects.all()
Question.objects.filter(id=1)
Question.objects.filter(question_text__startwith="What")     #question_text是models.CharField类
Question.objects.filter(publish_date__year=2016)   #publish_date是models.DateTimeField类
Question.objects.get(pk=1) #等价于Question.objects.get(id=1)

---

添加Choice

In [ ]:
a = Question.objects.get(pk=1)
a.choice_set.all()   #访问选项
-> []
a.choice_set.create(choice_text='Not much', votes=0) #添加一个选项
a.choice_set.create(choice_text='Good!', votes=0) #添加第二个选项
c = a.choice_set.create(choice_text='HaHa!', votes=0) #添加第三个选项

---

访问Choice对象的属性

In [ ]:
c.question
-> <Question: What's up?>

---

访问问题的所有选项

In [ ]:
a.choice_set.all()
-> [<Choice: Not much>, <Choice: Good!>, <Choice: HaHa!>]

---

问题的选项数目

In [ ]:
a.choice_set.count()
-> 3

---

直接访问(可参照上面的Question)

In [ ]:
Choice.objects.filter(question__publish_date__year=2016)
-> [<Choice: Not much>, <Choice: Good!>, <Choice: HaHa!>]

---

删除选项(删除问题也是一样的方法)

In [ ]:
c = a.choice_set.filter(choice_text__startwith='HaH')
c.delete()

---

# *4.可视化管理*

## 4.1创建一个管理员用户

In [ ]:
$ python manage.py createsuperuser
#输入用户名：
Username: XXXXX
#输入邮箱地址：
Email address: XXXXXX@XXXXX.com
#输入密码：
Password: XXXXXX      #不会显示
Password(again): XXXXXX

## 4.2启动开发服务器

In [ ]:
$ python manage.py runserver

进入：127.0.0.1:8000/admin  
输入账号密码

![Foo](http://python.usyiyi.cn/django/_images/admin01.png)

## 4.3注册poll应用

在polls/admin.py中添加要注册的内容

In [ ]:
from .models import Question
admin.site.register(Question)

## 4.4自定义标题排列顺序

在polls/admin.py中添加内容

In [ ]:
class QuestionAdmin(admin.ModelAdmin):
    fields = ['publish_date', 'question_text']
    #排列顺序为'publish_date', 'question_text'
admin.site.register(Question, QuestionAdmin)  #在注册时添加

![Foo](http://python.usyiyi.cn/django/_images/admin07.png)

## 4.5分组排列

In [ ]:
class QuestionAdmin(admin.ModelAdmin):
    fieldsets = [
        (None,               {'fields':['question_text']}),
        ('Date information', {'fields':['publish_date']}),
        ]  #Date information为小标题
    #排列顺序为'question_text', 'publish_date' 
admin.site.register(Question, QuestionAdmin)

![foo](http://python.usyiyi.cn/django/_images/admin08t.png)

### 4.5.1为字段集指定HTML样式类

In [ ]:
{'fields':['publish_date']， 'classes':['collapse']}  #可用于任何地方,collapse为可收放(show/hide)

![foo](http://python.usyiyi.cn/django/_images/admin09.png)

## 4.6添加关联对象

在polls/admin.py中添加内容

In [ ]:
#也可以直接注册，但Question和Choice分开不便于编辑
class ChoiceInline(admin.StackedInline):   #TabularInline为表格式的，StackedInline为堆叠的
    #添加关联对象， 使Choice在Question中编辑
    model = Choice  #关联的模块
    extra = 3   #默认提供3个Choice空间

#在QuestionAdmin下添加:
inlines = [ChoiceInline]

![foo](http://python.usyiyi.cn/django/_images/admin11t.png)

### 4.6.1修改Choice的样式

admin.StackedInline可以改成其它样式，如admin.TabularInline

![foo](http://python.usyiyi.cn/django/_images/admin12t.png)

## 4.7在Question列表中显示其它属性

![foo](http://python.usyiyi.cn/django/_images/admin04t.png)

In [ ]:
#在QuestionAdmin下添加:
list_display = ('question_text', 'publish_date', 'was_published_recently')

![foo](http://python.usyiyi.cn/django/_images/admin13t.png)

### 4.7.1修改属性显示的内容

在polls/models.py下添加方法属性

In [ ]:
def was_published_recently(self):
    return self.pub_date >= timezone.now() - datetime.timedelta(days=1)
was_published_recently.admin_order_field = 'pub_date'
was_published_recently.boolean = True
was_published_recently.short_description = 'Published recently?'

## 4.8添加侧边栏

In [ ]:
#在QuestionAdmin下添加：
list_filter = ['publish_date']
#侧边栏的内容会根据'publish_date'的字段类型而定

![foo](http://python.usyiyi.cn/django/_images/admin14t.png)

## 4.9添加搜索框

In [ ]:
#在QuestionAdmin下添加：
search_fields = ['question_text']
#搜索'question_text'的内容

## 4.10自定义模板

在项目的文件夹内创建一个templates文件夹  
打开项目配制文件(project01/settings.py)  

In [ ]:
#在TEMPLATES下的DIRS中添加
os.path.join(BASE_DIR, 'templates')

复制源模板文件(Lib\site-packages\django\contrib\admin\templates\admin)如base_site.html到(project01\templates\admin)中  
修改project01\templates\admin中的base_site.html即可。